In [112]:
import os
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import random
from matplotlib import pyplot as plt

# the following line is not required if Dataset is installed as a python package.
sys.path.append("../..")
from dataset import Pipeline, B, C, F, V
import mnist
from dataset.models.tf import ResNet18
from dataset import DatasetIndex, Dataset, Batch

from dataset import B, V, F, R, P, action, ImagesBatch
from dataset.models.tf import TFModel
from dataset.models.tf.layers import conv_block
from dataset.batch_image import transform_actions

In [41]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_labels = mnist.test_labels()

In [42]:
# 2 classes: 4 and others

four=[]
others=[]
for i in range(0, len(train_labels)):
    if train_labels[i]==4:
        four.append(i)
    else:
        others.append(i)
random.shuffle(others)

In [43]:
# new_mage_size 56 x 28
train_4 = train_images[four[0]]
train_4 = np.append(train_4, train_images[others[0]],axis=0)
zipped = zip(four, others[0:len(four)])
for i,j in zipped:
    if i != four[0]:
        train_4 = np.append(train_4, train_images[i], axis=0)
        train_4 = np.append(train_4, train_images[j], axis=0)
train_4 = train_4.reshape((len(four),56,28))
y_4 = np.ones((len(four)))

In [44]:
train_others = train_images[others[len(four)]]
for i in others[len(four):len(four)*3]:
    if i != others[len(four)]:
        train_others = np.append(train_others, train_images[i], axis=0)
train_others = train_others.reshape((len(four),56,28))
y_others = np.zeros((len(four)))

In [63]:
train_images_new = np.concatenate((train_4, train_others), axis=0)
train_labels_new = np.concatenate((y_4, y_others),axis=0)

In [66]:
config = dict(model=ResNet18)

In [83]:
@transform_actions(prefix='_', suffix='_', wrapper='apply_transform')
class MnistBatch(ImagesBatch):
    components = 'images', 'labels'

    def _shift_flattened_(self, image, max_margin=8):
        padded = np.pad(self._to_array_(image),
                        pad_width=[[max_margin, max_margin], [max_margin, max_margin], [0, 0]],
                        mode='minimum')
        left_lower = np.random.randint(2 * max_margin, size=2)
        slicing = (slice(left_lower[0], left_lower[0] + 28),
                   slice(left_lower[1], left_lower[1] + 28))
        return self._to_pil_(padded[slicing])

In [84]:
data = train_images_new, train_labels_new

In [103]:
train_template = (Pipeline(config=config)
                .init_variable('loss_history', init_on_each_run=list)
                .init_variable('current_loss', init_on_each_run=0)
                .shift_flattened()
                .init_model('dynamic', C('model'), 'conv_nn',
                            config={'inputs': dict(images={'shape': B('image_shape')},
                                                   labels={'classes': 2, 'transform': 'ohe', 'name': 'targets'}),
                                    'input_block/inputs': 'images',
                                    'output': dict(ops=['accuracy'])})
                .to_array()
                .train_model('conv_nn', fetches='loss',
                                     feed_dict={'images': B('images'),
                                                'labels': B('labels')},
                             save_to=V('current_loss'))
                .update_variable('loss_history', V('current_loss'), mode='a')
)

In [97]:
client_index = list(range(train_labels_new.shape[0]))
client_dataset = Dataset(client_index, batch_class=MnistBatch, preloaded=data)
client_dataset.cv_split([.8, .1, .1])

In [98]:
train_pipeline = (train_template << client_dataset.train)

In [99]:
BATCH_SIZE = 64
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, drop_last=True, bar=True)








  0%|          | 0/146 [00:00<?, ?it/s]

 25%|██▌       | 7085/28044 [17:37<52:09,  6.70it/s]




  6%|▌         | 1/18 [02:17<38:58, 137.55s/it]



  6%|▌         | 1/18 [02:43<46:11, 163.01s/it]






  1%|▏         | 2/146 [00:10<12:58,  5.40s/it]






  2%|▏         | 3/146 [00:13<11:05,  4.65s/it]






  3%|▎         | 4/146 [00:16<09:26,  3.99s/it]






  3%|▎         | 5/146 [00:18<08:31,  3.63s/it]






  4%|▍         | 6/146 [00:21<07:53,  3.38s/it]






  5%|▍         | 7/146 [00:24<07:22,  3.18s/it]






  5%|▌         | 8/146 [00:27<07:03,  3.07s/it]






  6%|▌         | 9/146 [00:30<07:03,  3.09s/it]






  7%|▋         | 10/146 [00:33<07:01,  3.10s/it]






  8%|▊         | 11/146 [00:36<06:39,  2.96s/it]






  8%|▊         | 12/146 [00:38<06:24,  2.87s/it]






  9%|▉         | 13/146 [00:42<06:38,  2.99s/it]






 10%|▉         | 14/146 [00:45<06:36,  3.00s/it]






 10%|█         | 15/146 [00:48<06:36,  3.03s/it]






 11%|█         | 16/146 [00:

In [133]:
test_pipeline = (client_dataset.test.p
                .import_model('conv_nn', train_pipeline)
                .init_variable('accuracy', init_on_each_run=list)
                .to_array()
                .predict_model('conv_nn', fetches='output_accuracy', 
                               feed_dict={'images': B('images'), 'labels': B('labels')},
                               save_to=V('accuracy'), mode='a')
                .run(BATCH_SIZE, shuffle=True, n_epochs=1, drop_last=True, bar=True)
)














  0%|          | 0/18 [00:00<?, ?it/s]












 11%|█         | 2/18 [00:00<00:02,  6.29it/s]












 17%|█▋        | 3/18 [00:00<00:03,  4.98it/s]












 22%|██▏       | 4/18 [00:00<00:03,  4.06it/s]












 28%|██▊       | 5/18 [00:01<00:03,  3.34it/s]












 33%|███▎      | 6/18 [00:01<00:03,  3.04it/s]












 39%|███▉      | 7/18 [00:02<00:03,  2.84it/s]












 44%|████▍     | 8/18 [00:02<00:03,  2.71it/s]












 50%|█████     | 9/18 [00:02<00:03,  2.75it/s]












 56%|█████▌    | 10/18 [00:03<00:02,  2.79it/s]












 61%|██████    | 11/18 [00:03<00:02,  2.85it/s]












 67%|██████▋   | 12/18 [00:03<00:02,  2.92it/s]












 72%|███████▏  | 13/18 [00:04<00:01,  2.91it/s]












 78%|███████▊  | 14/18 [00:04<00:01,  2.87it/s]












 83%|████████▎ | 15/18 [00:05<00:01,  2.88it/s]












 89%|████████▉ | 16/18 [00:05<00:00,  2.90it/s]












 94%|█████████▍| 17/18 [00:05<00:00,  2.98it

In [134]:
accuracy = np.array(test_pipeline.get_variable('accuracy')).mean()
print('Accuracy {:6.2f}'.format(accuracy))

Accuracy   0.99
